In [ ]:
import flask
import matplotlib.pyplot as plt

In [ ]:
import pandas
x = pandas.read_csv("data/train.csv")

In [ ]:
import plotly.graph_objects as go
import json
import plotly

x_val = ['Action Denied','Action Accepted']
colors = ['lightslategray','crimson'] 

x_0 = len(x[x['ACTION'] == 0])
x_1 = len(x[x['ACTION'] == 1])
fig = plotly.graph_objects.Figure([go.Bar(x=x_val, y = [x_0,x_1], text=[x_0,x_1],textposition='auto',  marker_color=colors)])
fig.update_layout(title_text='Count of Action Denied and Action Accepted - Clearly Class Imbalance!')
a = fig.to_json()
z = json.dumps(a,cls = plotly.utils.PlotlyJSONEncoder)

In [ ]:
unique = pandas.DataFrame([(col,x[col].nunique()) for col in x.columns], 
                           columns=['Columns', 'Count of Category'])
colors = ['lightslategray'] 

fig = plotly.graph_objects.Figure([go.Bar(x=unique.Columns, y = unique['Count of Category'], text=unique['Count of Category'],textposition='auto',  marker_color=colors)])
fig.update_layout(title_text='Unique categories - Train Data Set!')
fig.show()

In [ ]:
a = fig.to_json()
z = json.dumps(a,cls = plotly.utils.PlotlyJSONEncoder)

# Drop Duplicate Columns

In [ ]:
train = x

In [ ]:
import itertools
target = "ACTION"
col4train = [val for val in train.columns if val!=target]

col1 = 'ROLE_CODE'
col2 = 'ROLE_TITLE'

pair = len(train.groupby([col1,col2]).size())
single = len(train.groupby([col1]).size())

print(col1, col2, pair, single)



In [ ]:
import numpy as np

In [ ]:
a = set(x.ROLE_TITLE.unique())

In [ ]:
b = set(x.ROLE_CODE.unique())

In [ ]:
a.intersection(b)


In [ ]:
for i in a:
    if i in b:
        print(i)

In [ ]:
cols = x[['ROLE_CODE', 'ROLE_TITLE']]


In [ ]:
cols

In [ ]:
df = x

In [ ]:
import seaborn as sns
import io

In [ ]:
plotlist = []
fig = plt.figure(figsize=(30,20))
fig = sns.heatmap(df.corr(),annot=True, cmap ='viridis', linewidth = 1)



In [ ]:
len(plotlist)

In [ ]:
plotlist = []
fig = plt.figure()
fig = plt.figure(figsize=(30,20))
for i in range(1,10):
    fig =plt.subplot(5,2,i)
    fig = plt.hist(df[df.columns[i]])
    fig = plt.xlabel(df.columns[i])
    fig = plt.ylabel("Frequency")
buf = io.BytesIO()
fig.figure.savefig(buf, format = "png", dpi = 600, box_inches = "tight", pad_inches = 0)
plotlist.append(buf.getvalue())

In [ ]:

"""
Created on Sat May 23 12:25:13 2020

@author: gandhi
"""
import itertools
import numpy as np
import pandas as pd
import os
from catboost.datasets import amazon
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn.model_selection
from sklearn.model_selection import StratifiedKFold,cross_validate,train_test_split
from sklearn.metrics import classification_report,confusion_matrix, precision_recall_curve, roc_curve, roc_auc_score, log_loss
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE
from itertools import combinations
##Extracting the data
train_data, test_data =amazon()
train_data.head()
df= train_data
## Analysing the data
plt.figure()
plt.figure(figsize=(30,20))
for i in range(1,10):
    plt.subplot(5,2,i)
    plt.hist(df[df.columns[i]])
    plt.xlabel(df.columns[i])
    plt.ylabel("Frequency")
plt.show()

plt.figure(figsize=(30,20))
sns.heatmap(df.corr(),annot=True, cmap ='viridis', linewidth = 1)
plt.figure(figsize=(10,30))
sns.heatmap(df.corr(),annot=True,cmap='magma',linewidth=2)
## null data set
null_train = (train_data.isnull())
null_test =  test_data.isnull()
##Imbalanced Dataset graph
sns.countplot (train_data['ACTION'])
#duplicate columns
train_data.apply(lambda z: len(z.unique()))
##confirming the duplicated columns

for feature_1,feature_2 in combinations(train_data.columns, 2):
    condition1=len(train_data.groupby([feature_1,feature_2]).size())==len(train_data.groupby([feature_1]).size())
    condition2=len(train_data.groupby([feature_1,feature_2]).size())==len(train_data.groupby([feature_2]).size())
    condition3=(train_data[feature_1].nunique()==train_data[feature_2].nunique())
    if (condition1 | condition2) & condition3:
        print(feature_1,feature_2)
        print('Duplicate Data')
##drop the duplicated columns
new_train_data= train_data.drop(['ROLE_CODE'], axis =1)
Y = train_data["ACTION"].values
new_test_data = test_data.drop(['ROLE_CODE' ,'id'], axis =1)
##Sampling
##sampling 
X, y = make_classification(n_classes=2, class_sep=2, weights=[0.2, 0.8], n_informative=2, n_redundant=0, flip_y=0, n_features=8, n_clusters_per_class=1, n_samples=1000, random_state=10)
print('Original dataset shape %s' % Counter(y))
sm = SMOTE(random_state = 2) 
X_train_res, y_train_res = sm.fit_sample(new_train_data,Y)
train_data_sam = (X_train_res[X_train_res["ACTION"]==0]).index
sample_0 = len(X_train_res[X_train_res["ACTION"]==0])
train_data_sam1= X_train_res[X_train_res["ACTION"]==1].index
sample_1 = len(X_train_res[X_train_res["ACTION"]==0])
#train_random_0= np.random.choice(train_data_0, sample_0, replace=False) 
#train_0= train_data.loc[train_random_0]
#new_train_0 = train_0.drop('ROLE_CODE' , axis = 1)
#df_group_train_0 = new_train_0.groupby('ROLE_TITLE')['RESOURCE'].nunique()
##Test data 
## Encoding

target = 'ACTION'
train_columns = [x for x in X_train_res if x not in [target]]

##
def encode_dataset(train, test, func, func_params = {}):
    dataset = pd.concat([train, test], ignore_index = True)
    dataset = func(dataset, **func_params)
    if isinstance(dataset, pd.DataFrame):
        new_train = dataset.iloc[:train.shape[0],:].reset_index(drop = True)
        new_test =  dataset.iloc[train.shape[0]:,:].reset_index(drop = True)
    else:
        new_train = dataset[:train.shape[0]]
        new_test =  dataset[train.shape[0]:]
    return new_train, new_test
def one_hot(dataset):
    ohe = OneHotEncoder(sparse=True, dtype=np.float32, handle_unknown='ignore')
    return ohe.fit_transform(dataset.values)
new_train, new_test = encode_dataset(X_train_res[train_columns], new_test_data[train_columns], one_hot)

print(new_train.shape, new_test.shape)
# Split into train & validation set
X_train, X_val, y_train, y_val = train_test_split(new_train, Y, train_size=0.8)

X_train_un, X_val_un, y_train_un, y_val_un = train_test_split(new_train_data, Y, train_size=0.8)
##LOGISTIC REGRESSION
 
model = LogisticRegression (penalty='l2',  
                C=1.0, dual = False,
                fit_intercept=True, 
                random_state=500,
                solver = 'liblinear',
                max_iter = 1000,)
model.fit(X_train, y_train)
from sklearn.ensemble import RandomForestClassifier
model_1 = RandomForestClassifier(n_estimators = 99, criterion = 'entropy', random_state = 500)
model_1.fit(X_train_un, y_train_un)
##
def validation(model, X_test, y_test):
# Make predictions on test set
    y_pred=model.predict(X_test)
    y_pred=np.round(y_pred)
    
    # Confusion matrix
    print(confusion_matrix(y_test, y_pred))
    
    # AUC score
    y_pred_prob = model.predict_proba(X_test)
    print("AUC score: ", roc_auc_score(y_test, y_pred_prob[:,1]))
    

    # Accuracy, Precision, Recall, F1 score
    print(classification_report(y_test, y_pred))
    
    # Precision-Recall curve
    precisions, recalls, thresholds = precision_recall_curve(y_test, y_pred)
    plt.plot(recalls, precisions, "b-", linewidth=2)
    plt.xlabel("Recall", fontsize=16)
    plt.ylabel("Precision", fontsize=16)
    plt.axis([0, 1, 0, 1])
    plt.grid(True)
    plt.show()

    # ROC curve
    fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob[:,1])
    plt.plot([0, 1], [0, 1],'k--')
    plt.plot(fpr, tpr, label='Neural Network')
    plt.xlabel('False Positive')
    plt.ylabel('True Positive ')
    plt.title('ROC Curve')
    plt.show()
    
##
validation(model_1, X_val_un, y_val_un)
stats = cross_validate(model, new_train, Y, groups=None, scoring='roc_auc', cv=5, n_jobs=2, return_train_score = True)
stats = pd.DataFrame(stats)
stats.describe().transpose()
## grouping
column_values=new_train_data["RESOURCE"].values
unique_values = np.unique(column_values)
print(unique_values)
df_group_MGR = new_train.groupby('MGR_ID')['RESOURCE'].nunique()
df_group_Title = new_train.groupby('ROLE_TITLE')['RESOURCE'].nunique()
##features and target
from sklearn.model_selection import train_test_split
x_features = new_train.values.drop("ACTION", axis = 1)
y_target = new_train["ACTION"]
x_features_train, x_features_valid, y_target_train, y_target_val = StratifiedKFold(x_features, y_target, train_size=0.8)
## determinig all the features for action 0
train_data_0 = new_train[new_train["ACTION"]==0].index
sample_0 = len(new_train[new_train["ACTION"]==0])
train_random_0= np.random.choice(train_data_0, sample_0, replace=False) 
train_0= train_data.loc[train_random_0]
new_train_0 = train_0.drop('ROLE_CODE' , axis = 1)
df_group_train_0 = new_train_0.groupby('ROLE_TITLE')['RESOURCE'].nunique()
##determining all the features for action 1

train_data_1 = new_train[new_train["ACTION"]==1].index
sample_1 = len(new_train[new_train["ACTION"]==1])
train_random_1= np.random.choice(train_data_1, sample_1, replace=False) 
train_1= train_data.loc[train_random_1]
new_train_1 = train_1.drop('ROLE_CODE' , axis = 1)
df_group_train_1 = new_train_1.groupby('ROLE_TITLE')['RESOURCE'].nunique()
## Concatinatig both the colummns
conc_train = pd.concat([new_train_1,new_train_0], ignore_index=True)
len(conc_train)


##pre processed dataset
##machine learning on basic
#to apply sampling techniques
##FLASK
try3 = train_data ['ROLE_TITLE']
try2 = train_data['ROLE_CODE']['ROLE_TITLE']




In [ ]:
import pandas
data = pandas.read_csv("data/train.csv")

In [ ]:

from sklearn.model_selection import train_test_split 
  
X = data.iloc[:,1:]
y = data.iloc[:,:1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0) 
  
# describes info about train and test set 
print("Number transactions X_train dataset: ", X_train.shape) 
print("Number transactions y_train dataset: ", y_train.shape) 

print("Number transactions X_test dataset: ", X_test.shape) 
print("Number transactions y_test dataset: ", y_test.shape) 

sm = SMOTE(random_state = 2) 
X_train_res, y_train_res = sm.fit_sample(X_train, y_train.values.ravel()) 

print('before OverSampling, the shape of train_X: {}'.format(X_train.shape)) 
print('before OverSampling, the shape of train_y: {} \n'.format(y_train.shape)) 
  
print("before OverSampling, counts of label '1': {}".format(sum(y_train.values.ravel() == 1))) 
print("before OverSampling, counts of label '0': {}".format(sum(y_train.values.ravel() == 0))) 


print('After OverSampling, the shape of train_X: {}'.format(X_train_res.shape)) 
print('After OverSampling, the shape of train_y: {} \n'.format(y_train_res.shape)) 
  
print("After OverSampling, counts of label '1': {}".format(sum(y_train_res == 1))) 
print("After OverSampling, counts of label '0': {}".format(sum(y_train_res == 0))) 


In [ ]:
target = 'ACTION'
train_columns = [x for x in data if x not in [target]]

In [ ]:
import itertools
import numpy as np
import pandas as pd
import os
from catboost.datasets import amazon
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn.model_selection
from sklearn.model_selection import StratifiedKFold,cross_validate,train_test_split
from sklearn.metrics import classification_report,confusion_matrix, precision_recall_curve, roc_curve, roc_auc_score, log_loss
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE
from itertools import combinations
##Extracting the data
train_data, test_data =amazon()
train_data.head()
df= train_data

X= train_data.drop(['ROLE_CODE'], axis =1)
y = train_data["ACTION"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0) 
  
# describes info about train and test set 
print("Number transactions X_train dataset: ", X_train.shape) 
print("Number transactions y_train dataset: ", y_train.shape) 

print("Number transactions X_test dataset: ", X_test.shape) 
print("Number transactions y_test dataset: ", y_test.shape) 

sm = SMOTE(random_state = 2) 
X_train_res, y_train_res = sm.fit_sample(X_train, y_train.values.ravel()) 

print('before OverSampling, the shape of train_X: {}'.format(X_train.shape)) 
print('before OverSampling, the shape of train_y: {} \n'.format(y_train.shape)) 
  
print("before OverSampling, counts of label '1': {}".format(sum(y_train.values.ravel() == 1))) 
print("before OverSampling, counts of label '0': {}".format(sum(y_train.values.ravel() == 0))) 


print('After OverSampling, the shape of train_X: {}'.format(X_train_res.shape)) 
print('After OverSampling, the shape of train_y: {} \n'.format(y_train_res.shape)) 
  
print("After OverSampling, counts of label '1': {}".format(sum(y_train_res == 1))) 
print("After OverSampling, counts of label '0': {}".format(sum(y_train_res == 0))) 


target = 'ACTION'
train_columns = [x for x in X_train_res if x not in [target]]





In [20]:
len(X_train.columns)

9

In [11]:
import pandas as pd

In [12]:
df_new = pd.DataFrame(X_train_res, y_train_res)
df_new.columns = X_train.columns

In [24]:
df_new

,ACTION,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY
0,0,79121,6235,117961,118343,118437,307024,311622,118331
1,1,74883,17614,117902,118041,118042,118043,118044,270488
1,1,72168,24973,91261,118026,118229,273308,120072,249618
1,1,71236,5294,119596,119597,118979,118293,118294,118295
1,1,25882,5490,117961,117962,5488,120618,304465,118643
1,1,1020,3262,117961,118343,123125,118054,295130,117887
1,1,74909,4565,117961,118386,121961,118784,117906,290919
1,1,18552,8088,117961,118413,118481,118321,118448,290919
1,1,34576,3242,117961,118343,118514,118321,117906,290919
1,1,6977,873,117961,118413,120526,118321,117906,290919
